In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_parquet("../Dataset/NF-CSE-CIC-IDS2018/NF-CSE-CIC-IDS2018-V2_train.parquet")

In [3]:
df_valid = pd.read_parquet("../Dataset/NF-CSE-CIC-IDS2018/NF-CSE-CIC-IDS2018-V2_valid.parquet")

In [4]:
df_test = pd.read_parquet("../Dataset/NF-CSE-CIC-IDS2018/NF-CSE-CIC-IDS2018-V2_test.parquet")

In [5]:
load_X_df_train = df_train.drop(["Label"],axis=1)
Y_df_train = df_train["Label"]

In [6]:
load_X_df_valid = df_valid.drop(["Label"],axis=1)
Y_df_valid = df_valid["Label"]

In [7]:
load_X_df_test = df_test.drop(["Label"],axis=1)
Y_df_test = df_test["Label"]

In [8]:
from sklearn.preprocessing import QuantileTransformer

In [9]:
scaler_df = QuantileTransformer(output_distribution='normal')

In [10]:
X_df_train = scaler_df.fit_transform(load_X_df_train)

In [11]:
X_df_valid = scaler_df.fit_transform(load_X_df_valid)

In [12]:
X_df_test = scaler_df.fit_transform(load_X_df_test)

In [13]:
from keras.models import Sequential
from keras.layers import Dense

2025-11-17 17:07:27.105292: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-17 17:07:27.108707: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-17 17:07:27.115468: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-17 17:07:27.125726: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-17 17:07:27.128819: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-17 17:07:27.138226: I tensorflow/core/platform/cpu_feature_gu

In [14]:
from keras.metrics import Recall, Precision

In [15]:
def fit_model(trainX, trainy):
    model = Sequential(name="N4_model")
    model.add(Dense(9, input_dim=39, activation='relu', name="N4_i"))
    model.add(Dense(7, activation='relu', name="N4_l1"))
    model.add(Dense(5, activation='relu', name="N4_l2"))
    model.add(Dense(3, activation='relu', name="N4_l3"))
    model.add(Dense(1, activation='sigmoid', name="N4_o")) 
    model._name="N4_m"
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', Recall(), Precision()])
    
    model.fit(trainX, trainy, epochs=10, batch_size=1000, verbose=1, validation_data=(X_df_valid, Y_df_valid))
    return model

In [16]:
model = fit_model(X_df_train, Y_df_train)

filename = './model_N4.keras'
model.save(filename)
print('>Saved %s' % filename)

/home/awardana/miniconda3/envs/cids_env/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
10792/10792 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.9802 - loss: 0.0862 - precision: 0.9758 - recall: 0.8386 - val_accuracy: 0.9502 - val_loss: 0.1760 - val_precision: 0.7600 - val_recall: 0.8441
Epoch 2/10
10792/10792 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.9938 - loss: 0.0343 - precision: 0.9953 - recall: 0.9522 - val_accuracy: 0.9440 - val_loss: 0.2524 - val_precision: 0.7492 - val_recall: 0.7896
Epoch 3/10
10792/10792 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.9940 - loss: 0.0331 - precision: 0.9963 - recall: 0.9528 - val_accuracy: 0.9505 - val_loss: 0.1882 - val_precision: 0.7658 - val_recall: 0.8353
Epoch 4/10
10792/10792 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.9941 - loss: 0.0328 - precision: 0.9969 - recall: 0.9528 - val_accuracy: 0.9541 - val_loss: 0.2782 - val_precision: 0.7633 - val_recall: 0.8853
Epoch 5/10
10792/10792 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.9941 - loss: 0.0328 - precision: 0.9971 - recall: 0.9527 - val_accura

In [17]:
from keras.models import load_model
from numpy import argmax
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [18]:
filename_model = './model_N4.keras'
loaded_model = load_model(filename_model)

In [19]:
score = loaded_model.evaluate(X_df_test, Y_df_test, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test recall:', score[2])
print('Test precision:', score[3])

# Calculate F1-score
precision = score[3]
recall = score[2]
f1_score = 2 * (precision * recall) / (precision + recall)

print('Test F1-score:', f1_score)


144532/144532 ━━━━━━━━━━━━━━━━━━━━ 113s 781us/step - accuracy: 0.9026 - loss: 0.4341 - precision: 0.6484 - recall: 0.3874
Test loss: 0.43384355306625366
Test accuracy: 0.9025260806083679
Test recall: 0.3875586688518524
Test precision: 0.6476954221725464
Test F1-score: 0.48494370186983327
